<center>
    <p style="text-align:center">
        <img alt="Ragas" src="https://github.com/explodinggradients/ragas/blob/main/docs/_static/imgs/logo.png?raw=true" width="400">
        <br>
        <a href="https://docs.arize.com/phoenix/">Phoenix Docs</a>
        |
        <a href="https://github.com/explodinggradients/ragas">Ragas</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Evaluating a Search and Retrieval Application using Ragas</h1>

Imagine you're an engineer at Arize AI and you've built and deployed a documentation question-answering service using LlamaIndex. Users send questions about Arize's core product via a chat interface, and your service retrieves documents from your documentation in order to generate a response to the user. As the engineer in charge of evaluating and maintaining this system, you want to evaluate the quality of the responses from your service.

Phoenix helps you:
- identify gaps in your documentation
- detect queries for which the LLM gave bad responses
- detect failures to retrieve relevant context

In this tutorial, you will:

- Download an pre-indexed knowledge base of the Arize documentation and run a LlamaIndex application
- Visualize user queries and knowledge base documents to identify areas of user interest not answered by your documentation
- Find clusters of responses with negative user feedback
- Identify failed retrievals using cosine similarity, Euclidean distance, and RAG assessment metrics (RAGAS)

Parts of this notebook require an [OpenAI API key](https://platform.openai.com/account/api-keys) to run. 

## Chatbot Architecture

Your chatbot was built using LlamaIndex's low-level API. The architecture of your chatbot is shown below and can be explained in five steps.

![llama-index chatbot architecture](http://storage.googleapis.com/arize-phoenix-assets/assets/docs/notebooks/llama-index-knowledge-base-tutorial/llama_index_chatbot_architecture.png)

1. The user sends a query about Arize to your service.
1. `langchain.embeddings.OpenAIEmbeddings` makes a request to the OpenAI embeddings API to embed the user query using the `text-embedding-ada-002` model.
1. `llama_index.retrievers.RetrieverQueryEngine` does a similarity search against the entries of your index knowledge base for the two most similar pieces of context by cosine similarity.
1. `llama_index.indices.query.ResponseSynthesizer` generates a response by formatting the query and retrieved context into a single prompt and sending a request to OpenAI chat completions API with the `gpt-3.5-turbo`.
2. The response is returned to the user.

Phoenix makes your search and retrieval system *observable* by capturing the inputs and outputs of these steps for analysis, including:

- your query embeddings
- the retrieved context and similarity scores for each query
- the generated response that is return to the user

With that overview in mind, let's dive into the notebook.

## 1. Install Dependencies and Import Libraries

Install Phoenix, LlamaIndex, datasets, and Ragas (Retrieval Augmented Generation Assessment)

In [ ]:
!pip install "openai>=1" "arize-phoenix[llama-index]" gcsfs datasets "ragas>=0.0.22"

Import libraries.

In [ ]:
import os
import textwrap
from getpass import getpass

import numpy as np
import openai
import pandas as pd
import phoenix as px
from datasets import Dataset
from gcsfs import GCSFileSystem
from llama_index import (
    ServiceContext,
    StorageContext,
    load_index_from_storage,
)
from llama_index.callbacks import CallbackManager, OpenInferenceCallbackHandler
from llama_index.callbacks.open_inference_callback import as_dataframe
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.graph_stores.simple import SimpleGraphStore
from llama_index.llms import OpenAI

pd.set_option("display.max_colwidth", 1000)

## 2. Configure Your OpenAI API Key

💭 Configure your OpenAI API key.

In [ ]:
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Please enter your OpenAI API key: ")
openai.api_key = openai_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key

## 3. Download Your Knowledge Base

Download your pre-built index from cloud storage and instantiate your storage context.

In [ ]:
file_system = GCSFileSystem(project="public-assets-275721")
index_path = "arize-phoenix-assets/datasets/unstructured/llm/llama-index/arize-docs/index/"
storage_context = StorageContext.from_defaults(
    fs=file_system,
    persist_dir=index_path,
    graph_store=SimpleGraphStore(),  # prevents unauthorized request to GCS
)

Download and unzip a pre-built knowledge base index consisting of chunks of the Arize documentation.

## 4. Run Your Question-Answering Service

💭 Start a LlamaIndex application from your downloaded index. Use the `OpenInferenceCallbackHandler` to store your data in [OpenInference format](https://github.com/Arize-ai/open-inference-spec), an open standard for capturing and storing AI model inferences that enables production LLMapp servers to seamlessly integrate with LLM observability solutions such as Arize and Phoenix.

In [ ]:
callback_handler = OpenInferenceCallbackHandler()
service_context = ServiceContext.from_defaults(
    llm=OpenAI(model_name="gpt-3.5-turbo", temperature=0),
    embed_model=OpenAIEmbedding(model="text-embedding-ada-002"),
    callback_manager=CallbackManager(handlers=[callback_handler]),
)
index = load_index_from_storage(
    storage_context,
    service_context=service_context,
)
query_engine = index.as_query_engine()

💭 Ask a few questions of your question-answering service and view the responses.

In [ ]:
max_line_length = 80
for query in [
    "How do I get an Arize API key?",
    "Can I create monitors with an API?",
    "How do I need to format timestamps?",
    "What is the price of the Arize platform",
]:
    print("Query")
    print("=====")
    print()
    print(textwrap.fill(query, max_line_length))
    print()
    response = query_engine.query(query)
    print("Response")
    print("========")
    print()
    print(textwrap.fill(str(response), max_line_length))
    print()

## 5. Load Your Data Into Pandas Dataframes

To use Phoenix, you must load your data into pandas dataframes. 

💭 Your query data is saved in a buffer on the callback handler you defined in step 4. Load the data from the buffer into a dataframe.

In [ ]:
query_data_buffer = callback_handler.flush_query_data_buffer()
sample_query_df = as_dataframe(query_data_buffer)
sample_query_df

The columns of the dataframe are:

- **:id.id:**: the query ID
- **:timestamp.iso_8601:**: the time at which the query was made
- **:feature.text:prompt**: the query text
- **:feature.[float].embedding:prompt**: the embedding representation of the query
- **:prediction.text:response**: the final response presented to the user
- **:feature.[str].retrieved_document_ids:prompt**: the list of IDs of the retrieved documents
- **:feature.[float].retrieved_document_scores:prompt**: the lists of cosine similarities between the query and retrieved documents

The column names are in OpenInference format and describe the category, data type and intent of each column.

Running queries against a large dataset takes a long time. Download a dataframe containing query data.

In [ ]:
query_df = pd.read_parquet(
    "http://storage.googleapis.com/arize-phoenix-assets/datasets/unstructured/llm/llama-index/arize-docs/query_data_complete3.parquet",
)
query_df.head()

Next load your knowledge base into a dataframe.

In [ ]:
def storage_context_to_dataframe(storage_context: StorageContext) -> pd.DataFrame:
    """Converts the storage context to a pandas dataframe.

    Args:
        storage_context (StorageContext): Storage context containing the index
        data.

    Returns:
        pd.DataFrame: The dataframe containing the index data.
    """
    document_ids = []
    document_texts = []
    document_embeddings = []
    docstore = storage_context.docstore
    vector_store = storage_context.vector_store
    for node_id, node in docstore.docs.items():
        document_ids.append(node.hash)  # use node hash as the document ID
        document_texts.append(node.text)
        document_embeddings.append(np.array(vector_store.get(node_id)))
    return pd.DataFrame(
        {
            "document_id": document_ids,
            "text": document_texts,
            "text_vector": document_embeddings,
        }
    )


database_df = storage_context_to_dataframe(storage_context)
database_df = database_df.drop_duplicates(subset=["text"])
database_df.head()

The columns of your dataframe are:

- **document_id:** the ID of the chunked document
- **text:** the chunked text in your knowledge base
- **text_vector:** the embedding vector for the text, computed during the LlamaIndex build using "text-embedding-ada-002" from OpenAI

The query and database datasets are drawn from different distributions; the queries are short questions while the database entries are several sentences to a paragraph. The embeddings from OpenAI's "text-embedding-ada-002" capture these differences and naturally separate the query and context embeddings into distinct regions of the embedding space. When using Phoenix, you want to "overlay" the query and context embedding distributions so that queries appear close to their retrieved context in the Phoenix point cloud. To achieve this, we compute a centroid for each dataset that represents an average point in the embedding distribution and center the two distributions so they overlap.

In [ ]:
database_centroid = database_df["text_vector"].mean()
database_df["text_vector"] = database_df["text_vector"].apply(lambda x: x - database_centroid)

query_centroid = query_df[":feature.[float].embedding:prompt"].mean()
query_df[":feature.[float].embedding:prompt"] = query_df[":feature.[float].embedding:prompt"].apply(
    lambda x: x - query_centroid
)

## 6. Run Ragas Retrieval Evaluations

Cosine similarity and Euclidean distance are reasonable proxies for retrieval quality, but they don't always work perfectly. A novel idea is to use LLMs to measure retrieval quality by simply asking the LLM whether each retrieved document is relevant to the corresponding query. Let's use Ragas, a library designed specifically for this purpose, to run retrieval evaluations. 

Ragas measures your pipeline's performance against different dimensions

 - Faithfulness: measures the information consistency of the generated answer against the given context. If any claims are made in the answer that cannot be deduced from context is penalized.

 - Context Relevancy: measures how relevant retrieved contexts are to the question. Ideally, the context should only contain information necessary to answer the question. The presence of redundant information in the context is penalized.

 - Answer Relevancy: refers to the degree to which a response directly addresses and is appropriate for a given question or context. This does not take the factuality of the answer into consideration but rather penalizes the present of redundant information or incomplete answers given a question.

 - Aspect Critiques: Designed to judge the submission against defined aspects like harmlessness, correctness, etc. You can also define your own aspect and validate the submission against your desired aspect. The output of aspect critiques is always binary.

The final ragas_score is the harmonic mean of individual metric scores. For details check out their [docs](https://github.com/explodinggradients/ragas)

In [ ]:
# Construct a hugging face dataset to run ragas

question = query_df[":feature.text:prompt"].tolist()
answer = query_df[":prediction.text:response"].tolist()
list_of_document_id_lists = query_df[":feature.[str].retrieved_document_ids:prompt"].tolist()
document_id_to_text = dict(zip(database_df["document_id"].to_list(), database_df["text"].to_list()))
contexts = [
    [document_id_to_text[doc_id] for doc_id in id_list] for id_list in list_of_document_id_lists
]

dataset_to_evaluate = Dataset.from_dict(
    {
        "question": question,
        "contexts": contexts,
        "answer": answer,
    }
)

In [ ]:
# Let's run Ragas
from ragas import evaluate
from ragas.metrics import answer_relevancy, context_relevancy, faithfulness

ragas_results = evaluate(
    dataset_to_evaluate, metrics=[faithfulness, answer_relevancy, context_relevancy]
)

In [ ]:
# Let's take a look at the results
evals_df = ragas_results.to_pandas()
evals_df.head()

In [ ]:
# Let's augment the query_df with the ragas calculated metrics
query_df[":tag.float:answer_relevancy"] = evals_df["answer_relevancy"]
query_df[":tag.float:context_relevancy"] = evals_df["context_relevancy"]
query_df[":tag.float:faithfulness"] = evals_df["faithfulness"]

## 7. Launch Phoenix

Define your knowledge base dataset with a schema that specifies the meaning of each column (features, predictions, actuals, tags, embeddings, etc.). See the [docs](https://docs.arize.com/phoenix/) for guides on how to define your own schema and API reference on `phoenix.Schema` and `phoenix.EmbeddingColumnNames`.

In [ ]:
# get a random sample of 500 documents (including retrieved documents)
# this will be handled by by the application in a coming release
num_sampled_point = 500
retrieved_document_ids = set(
    [
        doc_id
        for doc_ids in query_df[":feature.[str].retrieved_document_ids:prompt"].to_list()
        for doc_id in doc_ids
    ]
)
retrieved_document_mask = database_df["document_id"].isin(retrieved_document_ids)
num_retrieved_documents = len(retrieved_document_ids)
num_additional_samples = num_sampled_point - num_retrieved_documents
unretrieved_document_mask = ~retrieved_document_mask
sampled_unretrieved_document_ids = set(
    database_df[unretrieved_document_mask]["document_id"]
    .sample(n=num_additional_samples, random_state=0)
    .to_list()
)
sampled_unretrieved_document_mask = database_df["document_id"].isin(
    sampled_unretrieved_document_ids
)
sampled_document_mask = retrieved_document_mask | sampled_unretrieved_document_mask
sampled_database_df = database_df[sampled_document_mask]

In [ ]:
database_schema = px.Schema(
    prediction_id_column_name="document_id",
    prompt_column_names=px.EmbeddingColumnNames(
        vector_column_name="text_vector",
        raw_data_column_name="text",
    ),
)
database_ds = px.Dataset(
    dataframe=sampled_database_df,
    schema=database_schema,
    name="database",
)

Define your query dataset. Because the query dataframe is in OpenInference format, Phoenix is able to infer the meaning of each column without a user-defined schema by using the `phoenix.Dataset.from_open_inference` class method.

In [ ]:
query_ds = px.Dataset.from_open_inference(query_df)

Launch Phoenix. Follow the instructions in the cell output to open the Phoenix UI.

In [ ]:
session = px.launch_app(primary=query_ds, corpus=database_ds)

## 8. Surface Problematic Clusters and Data Points

Phoenix helps you:

- visualize your embeddings
- color the resulting point cloud using evaluation metrics
- cluster the points and surface up problematic clusters based on whatever metric you care about

Congrats! You've identified a problematic cluster of queries. You now have tools at your disposal to investigate clusters of queries where your search and retrieval application is performing poorly based on:

- query percentage
- user feedback
- Ragas - RAG Assessment metrics
